In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104925/Novak-Djokovic&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2025 Madrid R64
L vsArnaldi
28.3%
35.8%
48.6%
33.4%
45.5%
54.5%
50.0%
53.6%
38.5%
45.0%
50.0%
0.0%
5.9%
47.1%
-118
2025 Monte Carlo R32
L vsTabilo
14.3%
23.2%
50.0%
19.8%
23.3%
40.0%
56.5%
33.7%
56.3%
42.9%
25.0%
4.2%
4.2%
43.5%
-76
2025 Miami F
L vsMensik
28.4%
41.9%
59.6%
35.9%
33.9%
51.8%
64.9%
43.9%
57.7%
66.7%
0.0%
11.8%
11.8%
46.7%
4
2025 Miami SF
W vsDimitrov
33.3%
44.4%
62.1%
38.9%
35.9%
48.7%
68.0%
42.3%
42.1%
45.0%
0.0%
20.0%
20.0%
25.0%
65
2025 Miami QF
W vsKorda
47.5%
59.3%
61.3%
55.4%
53.1%
67.3%
65.2%
62.7%
42.9%
57.1%
-
20.0%
20.0%
50.0%
-79
2025 Miami R16
W vsMusetti
19.6%
34.8%
63.9%
30.2%
28.1%
43.8%
65.2%
43.4%
46.7%
52.9%
-
0.0%
15.4%
61.5%
-118
2025 Miami R64
W vsHijikata
38.8%
57.1%
75.9%
48.6%
43.6%
59.0%
72.7%
55.9%
57.9%
45.0%
0.0%
22.2%
55.6%
85.7%
72
2025 Indian Wells R64
L vsVan De Zandschulp
17.8%
24.7%
43.9%
24.7%
24.5%
32.7%
51.4%
34.7%
60.0%
29.2%
40.0%
4.8%
9.5%
30.0%
-72
2025 Doha R32
L vsBerrettini
23.1%
40.0%
59.2%
34.5%
31.9%
53.2%
78.1%
47.7%
50.0%

In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,28.3%,35.8%,48.6%,33.4%,45.5%,54.5%,50.0%,53.6%,38.5%,45.0%,50.0%,0.0%,5.9%,47.1%,-118,2025,Madrid,R64,L,Arnaldi
1,14.3%,23.2%,50.0%,19.8%,23.3%,40.0%,56.5%,33.7%,56.3%,42.9%,25.0%,4.2%,4.2%,43.5%,-76,2025,Monte Carlo,R32,L,Tabilo
2,28.4%,41.9%,59.6%,35.9%,33.9%,51.8%,64.9%,43.9%,57.7%,66.7%,0.0%,11.8%,11.8%,46.7%,4,2025,Miami,F,L,Mensik
3,33.3%,44.4%,62.1%,38.9%,35.9%,48.7%,68.0%,42.3%,42.1%,45.0%,0.0%,20.0%,20.0%,25.0%,65,2025,Miami,SF,W,Dimitrov
4,47.5%,59.3%,61.3%,55.4%,53.1%,67.3%,65.2%,62.7%,42.9%,57.1%,-,20.0%,20.0%,50.0%,-79,2025,Miami,QF,W,Korda
5,19.6%,34.8%,63.9%,30.2%,28.1%,43.8%,65.2%,43.4%,46.7%,52.9%,-,0.0%,15.4%,61.5%,-118,2025,Miami,R16,W,Musetti
6,38.8%,57.1%,75.9%,48.6%,43.6%,59.0%,72.7%,55.9%,57.9%,45.0%,0.0%,22.2%,55.6%,85.7%,72,2025,Miami,R64,W,Hijikata
7,17.8%,24.7%,43.9%,24.7%,24.5%,32.7%,51.4%,34.7%,60.0%,29.2%,40.0%,4.8%,9.5%,30.0%,-72,2025,Indian Wells,R64,L,Van
8,23.1%,40.0%,59.2%,34.5%,31.9%,53.2%,78.1%,47.7%,50.0%,52.0%,66.7%,0.0%,5.9%,23.5%,-118,2025,Doha,R32,L,Berrettini
9,27.8%,35.2%,53.8%,29.6%,45.2%,51.6%,41.2%,48.4%,58.3%,63.2%,100.0%,4.3%,13.0%,63.6%,-79,2025,Australian Open,SF,L,Zverev


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/djokovic_serve_chart.csv', index=False) #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104925/Novak-Djokovic&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2024 US Open R32
L vsPopyrin
108.0
114.9
6.3
117.7
112.9
124
94
94.8
9.5
104.6
86.0
117
77
2024 US Open R64
W vsDjere
106.0
117.9
4.9
118.3
116.0
125
105
93.2
10.0
103.6
97.6
119
78
2024 US Open R128
W vsAlbot
107.5
116.6
6.6
119.9
110.8
126
104
96.7
11.0
94.8
101.2
117
78
2024 Wimbledon F
L vsAlcaraz
113.7
118.7
4.4
120.7
117.0
126
107
100.1
6.9
103.6
101.7
114
90
2024 Wimbledon SF
W vsMusetti
111.5
118.6
4.9
120.3
115.9
127
95
95.1
9.0
98.6
100.3
117
83
2024 Wimbledon R16
W vsRune
112.5
118.6
5.0
119.7
117.3
125
103
100.9
7.3
102.3
99.3
114
90
2024 Wimbledon R32
W vsPopyrin
112.4
117.6
4.5
118.9
116.3
125
105
99.9
9.0
103.5
95.0
115
76
2024 Wimbledon R64
W vsFearnley
111.9
116.3
4.6
118.1
113.4
123
106
102.3
8.0
108.1
98.3
115
88
2024 Wimbledon R128
W vsKopriva
109.8
114.9
4.9
118.3
111.3
123
105
98.2
8.8
105.5
104.5
112
85
2023 US Open F
W vsMedvedev
103.7
116.0
8.4
120.4
108.4
126
99
87.2
5.3
94.3
91.4
107
79
2023 US Open SF
W vsShelton
109.5
116.3
8.7
120.1
109.5
125
83
93.7
6.4
9

In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,108.0,114.9,6.3,117.7,112.9,124,94,94.8,9.5,104.6,86.0,117,77,2024,US Open,R32,L,Popyrin
1,106.0,117.9,4.9,118.3,116.0,125,105,93.2,10.0,103.6,97.6,119,78,2024,US Open,R64,W,Djere
2,107.5,116.6,6.6,119.9,110.8,126,104,96.7,11.0,94.8,101.2,117,78,2024,US Open,R128,W,Albot
3,113.7,118.7,4.4,120.7,117.0,126,107,100.1,6.9,103.6,101.7,114,90,2024,Wimbledon,F,L,Alcaraz
4,111.5,118.6,4.9,120.3,115.9,127,95,95.1,9.0,98.6,100.3,117,83,2024,Wimbledon,SF,W,Musetti
5,112.5,118.6,5.0,119.7,117.3,125,103,100.9,7.3,102.3,99.3,114,90,2024,Wimbledon,R16,W,Rune
6,112.4,117.6,4.5,118.9,116.3,125,105,99.9,9.0,103.5,95.0,115,76,2024,Wimbledon,R32,W,Popyrin
7,111.9,116.3,4.6,118.1,113.4,123,106,102.3,8.0,108.1,98.3,115,88,2024,Wimbledon,R64,W,Fearnley
8,109.8,114.9,4.9,118.3,111.3,123,105,98.2,8.8,105.5,104.5,112,85,2024,Wimbledon,R128,W,Kopriva
9,103.7,116.0,8.4,120.4,108.4,126,99,87.2,5.3,94.3,91.4,107,79,2023,US Open,F,W,Medvedev


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2024,US Open,R32,L,Popyrin,108.0,114.9,6.3,117.7,112.9,124,94,94.8,9.5,104.6,86.0,117,77
1,2024,US Open,R64,W,Djere,106.0,117.9,4.9,118.3,116.0,125,105,93.2,10.0,103.6,97.6,119,78
2,2024,US Open,R128,W,Albot,107.5,116.6,6.6,119.9,110.8,126,104,96.7,11.0,94.8,101.2,117,78
3,2024,Wimbledon,F,L,Alcaraz,113.7,118.7,4.4,120.7,117.0,126,107,100.1,6.9,103.6,101.7,114,90
4,2024,Wimbledon,SF,W,Musetti,111.5,118.6,4.9,120.3,115.9,127,95,95.1,9.0,98.6,100.3,117,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2011,Australian Open,QF,W,Berdych,110.7,118.9,9.3,NaN,NaN,128,88,93.3,8.6,NaN,NaN,108,75
242,2011,Australian Open,R16,W,Almagro,111.0,116.7,11.8,NaN,NaN,128,83,91.1,7.6,NaN,NaN,105,82
243,2011,Australian Open,R32,W,Troicki,115.8,121.1,4.7,NaN,NaN,128,109,99.0,8.5,NaN,NaN,112,88
244,2011,Australian Open,R64,W,Dodig,110.1,117.0,8.8,NaN,NaN,126,90,95.5,8.4,NaN,NaN,115,83


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [18]:
speed_df.to_csv('../data/djokovic_serve_speed.csv', index=False) #change the name of the file to save it as you want